In [1]:
# imports
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
base_url = "https://www.basketball-reference.com/leagues/NBA_"
boxscore_url = "https://www.basketball-reference.com/"
# https://www.basketball-reference.com/leagues/NBA_2022_games-november.html
years = range(2000, 2022, 1)
months = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]

In [9]:
years = [2022]
months = ["november"]
for year in years:
    for month in months:
        try:
            page = requests.get(base_url + str(year) + "_games-" + month + ".html")
            soup = BeautifulSoup(page.content, 'html.parser')
        except:
            print("month and year combo don't match")
        
        try:
            games = soup.find_all(lambda tag: tag.name == 'td' and tag.get('class') == ['center'])
        except:
            print("something went wrong with getting the games")
        for game in games
            try:
                boxscore = requests.get(boxscore_url + game.a['href'])
                boxsoup = BeautifulSoup(boxscore.content, 'html.parser')
            except:
                print("something went wrong with getting the box scores")


<class 'str'>
